<a href="https://colab.research.google.com/github/Sona30k/ML/blob/main/scrape_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bs4

In [ ]:
pip install requests

In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [36]:

def get_title(soup):

    try:

        title = soup.find("span", attrs={"id":'productTitle'})


        title_value = title.text


        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string


def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:

            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price


def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

if __name__ == '__main__':


    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})


    URLs = ["https://www.amazon.in/s?k=earphones&crid=1BOVF5QWC64HN&sprefix=earpho%2Caps%2C381&ref=nb_sb_noss_2",
    "https://www.amazon.in/s?k=beauty&crid=CTR9XBMKBPP0&sprefix=beauty%2Caps%2C402&ref=nb_sb_noss_2"
           ]
    all_data=[]
    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    for URL in URLs:
        print(f"Scraping: {URL}")
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "html.parser")
        links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})
        #links_list = []
        links_list = ["https://www.amazon.in" + link.get('href') for link in links if link.get('href')]

        for link in links_list[:10]:
          print(f"Fetching: {link}")
          product_page = requests.get(link, headers=HEADERS)
          product_soup = BeautifulSoup(product_page.content, "html.parser")
          all_data.append({
                "title": get_title(product_soup),
                "price": get_price(product_soup),
                "rating": get_rating(product_soup),
                "reviews": get_review_count(product_soup),
                "availability": get_availability(product_soup)
            })
          time.sleep(5)



    amazon_df = pd.DataFrame(all_data)


    amazon_df['title'] = amazon_df['title'].replace('', pd.NA)
    amazon_df.dropna(subset=['title'], inplace=True)

    # Save to a single CSV file
    amazon_df.to_csv("amazon_data.csv", mode='a', header=not pd.io.common.file_exists("amazon_data.csv"), index=False)
    print("Scraping completed! Data saved to amazon_data.csv")



Scraping: https://www.amazon.in/s?k=earphones&crid=1BOVF5QWC64HN&sprefix=earpho%2Caps%2C381&ref=nb_sb_noss_2
Fetching: https://www.amazon.in/boAt-BassHeads-100-Headphones-Black/dp/B071Z8M4KX/ref=sr_1_1?crid=1BOVF5QWC64HN&dib=eyJ2IjoiMSJ9.fbRW9FF9vLIWGzhk1v7QJdP7eZnf6DTL5Ag5de62N6q47Yfy0O_7_hgY-19o2bNseCVpLUkS1zhKVwTbNEM4e8GxVBvlHslJK4G3-wO3Rrg9qiAVobFTy-AScDN2kUy6UUGXAegdoxThcfyeExSFZwwmA3WCv8MtkkeTt3GAnlKTmj1j6Ucj613j-3_7r-7IPPvW1zhysxuaAX0mCLtljdEZNjSJH7bDFTp-9dUWEbE.DYQ8KxgVfbvotne8pmootkapZlUpPXeJx_sPmNpvt8I&dib_tag=se&keywords=earphones&qid=1743336379&sprefix=earpho%2Caps%2C381&sr=8-1
Fetching: https://www.amazon.in/Oneplus-Bluetooth-Wireless-Earphones-Bombastic/dp/B09TVVGXWS/ref=sr_1_2?crid=1BOVF5QWC64HN&dib=eyJ2IjoiMSJ9.fbRW9FF9vLIWGzhk1v7QJdP7eZnf6DTL5Ag5de62N6q47Yfy0O_7_hgY-19o2bNseCVpLUkS1zhKVwTbNEM4e8GxVBvlHslJK4G3-wO3Rrg9qiAVobFTy-AScDN2kUy6UUGXAegdoxThcfyeExSFZwwmA3WCv8MtkkeTt3GAnlKTmj1j6Ucj613j-3_7r-7IPPvW1zhysxuaAX0mCLtljdEZNjSJH7bDFTp-9dUWEbE.DYQ8KxgVfbvotne8pmootkapZlU

In [37]:
amazon_df

,title,price,rating,reviews,availability
0,boAt Bassheads 100 in Ear Wired Earphones with...,,4.3 out of 5 stars,"416,433 ratings",In stock
1,Oneplus Bullets Z2 Bluetooth Wireless in Ear E...,,4.1 out of 5 stars,"188,403 ratings",In stock
2,"ZEBRONICS Bro in Ear Wired Earphones with Mic,...",,4.5 out of 5 stars,"57,777 ratings",In stock
3,ZEBRONICS Zeb-Bro in Ear Wired Earphones with ...,,4.6 out of 5 stars,"57,777 ratings",In stock
4,"JBL C100SI Wired In Ear Headphones with Mic, J...",,4.1 out of 5 stars,"198,287 ratings",In stock
5,"ZEBRONICS Bro in Ear Wired Earphones with Mic,...",,4.4 out of 5 stars,"57,777 ratings",In stock
6,boAt BassHeads 100 C Wired Earphones with Type...,,4.6 out of 5 stars,"416,433 ratings",In stock
7,boAt Bassheads 152 in Ear Wired Earphones with...,,4.3 out of 5 stars,"78,420 ratings",In stock
8,Ambrane Wired in Ear Earphones with in-line Mi...,,4.6 out of 5 stars,"5,086 ratings",In stock
9,Boult Audio BassBuds X1 in-Ear Wired Earphones...,,4.3 out of 5 stars,"88,286 ratings",In stock
